In [ ]:
! pip install tensorflow

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Dropout, Activation, BatchNormalization, LeakyReLU
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import initializers
from tensorflow.keras.layers import add, concatenate
%matplotlib inline  
import warnings
warnings.filterwarnings('ignore')
from keras.callbacks import EarlyStopping, ModelCheckpoint
from scipy.sparse import csr_matrix
import tensorflow as tf
from tensorflow.python.keras.models import model_from_json
from sklearn import preprocessing
from keras.utils import plot_model

In [3]:
def dataPreprocessor(rating_df, num_users, num_items, init_value=0, average=False):
    """
        INPUT: 
            data: pandas DataFrame. columns=['userID', 'itemID', 'rating' ...]
            num_row: int. number of users
            num_col: int. number of items
            
        OUTPUT:
            matrix: 2D numpy array. 
    """
    if average:
      matrix = np.full((num_users, num_items), 0.0)
      for (_, userID, itemID, rating, timestamp) in rating_df.itertuples():
        matrix[userID, itemID] = rating
      avergae = np.true_divide(matrix.sum(1), np.maximum((matrix!=0).sum(1), 1))
      inds = np.where(matrix == 0)
      matrix[inds] = np.take(avergae, inds[0])
      
    else:
      matrix = np.full((num_users, num_items), init_value)
      for (_, userID, itemID, rating, timestamp) in rating_df.itertuples():
        matrix[userID, itemID] = rating

    return matrix

In [4]:
def show_error(history, skip):
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    plt.plot(np.arange(skip, len(loss), 1), loss[skip:])
    plt.plot(np.arange(skip, len(loss), 1), val_loss[skip:])
    plt.title('model train vs validation loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='best')
    plt.show()
     

def show_rmse(history, skip):
    rmse = history.history['masked_rmse_clip']
    val_rmse = history.history['val_masked_rmse_clip']
    plt.plot(np.arange(skip, len(rmse), 1), rmse[skip:])
    plt.plot(np.arange(skip, len(val_rmse), 1), val_rmse[skip:])
    plt.title('model train vs validation masked_rmse')
    plt.ylabel('rmse')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='best')
    plt.show()
     
def masked_se(y_true, y_pred):
        print(type(y_true),' ',type(y_pred))
        # masked function
        mask_true = K.cast(K.not_equal(y_true, 0), K.floatx())
        # masked squared error
        masked_squared_error = K.square(mask_true * (y_true - y_pred))
        masked_mse = K.sum(masked_squared_error, axis=-1)
        return masked_mse
def masked_se_clip_10(y_true, y_pred):
        print(type(y_true),' ',type(y_pred))
        # masked function
        mask_true = K.cast(K.not_equal(y_true, 0), K.floatx())
        y_pred = K.clip(y_pred, 1, 10)
        # masked squared error
        masked_squared_error = K.square(mask_true * (y_true - y_pred))
        masked_mse = K.sum(masked_squared_error, axis=-1)
        return masked_mse     

def masked_mse(y_true, y_pred):
        # masked function
        y_true = tf.cast(y_true, tf.float32)
        mask_true = K.cast(K.not_equal(y_true, 0), K.floatx())
        # masked squared error
        masked_squared_error = K.square(mask_true * (y_true - y_pred))
        masked_mse = K.sum(masked_squared_error, axis=-1) / K.maximum(K.sum(mask_true, axis=-1), 1)
        return masked_mse
     

def masked_rmse(y_true, y_pred):
        # masked function
        mask_true = K.cast(K.not_equal(y_true, 0), K.floatx())
        # masked squared error
        masked_squared_error = K.square(mask_true * (y_true - y_pred))
        masked_mse = K.sqrt(K.sum(masked_squared_error, axis=-1) / K.maximum(K.sum(mask_true, axis=-1), 1))
        return masked_mse

def masked_mse_clip_10(y_true, y_pred):
        # masked function
        y_true = tf.cast(y_true, tf.float32)
        mask_true = K.cast(K.not_equal(y_true, 0), K.floatx())
        y_pred = K.clip(y_pred, 1, 10)
        # masked squared error
        masked_squared_error = K.square(mask_true * (y_true - y_pred))
        masked_mse = K.sum(masked_squared_error, axis=-1) / K.maximum(K.sum(mask_true, axis=-1), 1)
        return masked_mse
def masked_rmse_clip_10(y_true, y_pred):
        # masked function
        mask_true = K.cast(K.not_equal(y_true, 0), K.floatx())
        y_pred = K.clip(y_pred, 1, 10)
        # masked squared error
        masked_squared_error = K.square(mask_true * (y_true - y_pred))
        masked_mse = K.sqrt(K.sum(masked_squared_error, axis=-1) / K.maximum(K.sum(mask_true, axis=-1), 1))
        return masked_mse

In [5]:
datContent = [i.strip().split() for i in open("anime_ratings.dat").readlines()][1:]

In [6]:
max_user=0
max_anime=0
for r in datContent:
  # print(r)
  max_user=max(max_user,int(r[0]))
  max_anime=max(max_anime,int(r[1]))
print(max_user,max_anime)

5000 7390


In [7]:
def my_proc(cont,init):
  matrix = np.full((max_user,max_anime),init)
  for r in cont:
    matrix[int(r[0])-1, int(r[1])-1] = float(r[2])
  return matrix

def my_proc_avg(cont): 
  matrix = np.full((max_user,max_anime),0)
  for r in cont:
    matrix[int(r[0])-1, int(r[1])-1] = float(r[2])
  avergae = np.true_divide(matrix.sum(1), np.maximum((matrix!=0).sum(1), 1))
  inds = np.where(matrix == 0)
  matrix[inds] = np.take(avergae, inds[0])
  return matrix

In [8]:
train_df, test_df = train_test_split(datContent,test_size=0.1,random_state=999613182)  
train_df, validate_df = train_test_split(train_df,test_size=0.1,random_state=999613182) 
print(len(train_df),len(test_df),len(validate_df))

340153 41995 37795


In [10]:
val=0
users_items_matrix_train_zero = my_proc(train_df,val)
users_items_matrix_train_avg = my_proc_avg(train_df)
users_items_matrix_validate = my_proc(validate_df,val)
users_items_matrix_test = my_proc(test_df,val)

In [11]:
users_items_matrix_train_zero.shape

(5000, 7390)

In [12]:
 def Deep_AE_model(X, layers, activation, last_activation, dropout, regularizer_encode, regularizer_decode, side_infor_size=0):
    '''
    Build Deep AE for CF
        INPUT: 
            X: #_user X #_item matrix
            layers: List, each element is the number of neuron for a layer
            reg: L2 regularization parameter
            activation: activation function for all dense layer except the last
            last_activation: activation function for the last dense layer
            dropout: dropout rate
            regularizer_encode: regularizer for encoder
            regularizer_decode: regularizer for decoder
            side_infor_size: size of the one hot encoding side information
        OUTPUT:
            Keras model
    '''

    # Input
    input_layer = x = Input(shape=(X.shape[1],), name='UserRating')
    
    # Encoder
    # -----------------------------
    k = int(len(layers)/2)
    i = 0
    for l in layers[:k]:
      x = Dense(l, activation=activation,
                      name='EncLayer{}'.format(i), kernel_regularizer=regularizers.l2(regularizer_encode))(x)
      i = i+1
      
      
    # Latent Space
    # -----------------------------
    x = Dense(layers[k], activation=activation, 
                                name='LatentSpace', kernel_regularizer=regularizers.l2(regularizer_encode))(x)
    
    # Dropout
    x = Dropout(rate = dropout)(x)
    
    # Decoder
    # -----------------------------
    for l in layers[k+1:]:
      i = i-1
      x = Dense(l, activation=activation, 
                      name='DecLayer{}'.format(i), kernel_regularizer=regularizers.l2(regularizer_decode))(x)
      
    # Output

    output_layer = Dense(X.shape[1]-side_infor_size, activation=last_activation, name='UserScorePred', kernel_regularizer=regularizers.l2(regularizer_decode))(x)

    # this model maps an input to its reconstruction
    model = Model(input_layer, output_layer)

    return model
     

In [13]:
# layers = [256, 512, 256]
layers = [512, 256, 128, 256, 512]
#layers = [512, 256, 512]
#layers = [128, 256, 512, 256, 128]
#layers = [512, 512, 512]
dropout = 0.8
# activation = 'sigmoid'
# last_activation = 'linear'
activation = 'selu'
last_activation = 'selu'
regularizer_encode = 0.001
regularizer_decode = 0.001

#Fold Validation

In [16]:
import numpy as np
from sklearn.model_selection import KFold
import gc

kf = KFold(n_splits=5,shuffle=True,random_state=999613182)
folds=[] #train validate, test
for train_idx, rem_idx in kf.split(datContent):
  train=[]
  validate=[]
  test=[]
  n = len(rem_idx)//2
  valid_idx=rem_idx[0:n]
  test_idx=rem_idx[n:]
  for i in train_idx:
    train.append(datContent[i])
  for i in valid_idx:
    validate.append(datContent[i])
  for i in test_idx:
    test.append(datContent[i])
  fold=[train,validate,test]
  print(len(fold[0]),len(fold[1]),len(fold[2]))
  folds.append(fold)

335954 41994 41995
335954 41994 41995
335954 41994 41995
335955 41994 41994
335955 41994 41994


In [17]:
f=0
for fold in folds:
  f+=1
  gc.collect()
  init_v = 0
  users_items_matrix_train_zero = my_proc(fold[0],init_v)
  users_items_matrix_train_avg = my_proc_avg(fold[0])
  users_items_matrix_validate = my_proc(fold[1],init_v)
  users_items_matrix_test = my_proc(fold[2],init_v)
  layers=[512]*11
  layers[5]=1024
  Deep_AE = Deep_AE_model(users_items_matrix_train_zero,layers, 'tanh', 'selu', 0.65, 0.001, 0.001)
  Deep_AE.compile(optimizer = Adam(lr=0.0001), loss=masked_mse, metrics=[masked_rmse,masked_rmse_clip_10])
  hist_Deep_AE = Deep_AE.fit(x=users_items_matrix_train_zero, y=users_items_matrix_train_zero,
                  epochs=100,
                  batch_size=256,
                  validation_data=[users_items_matrix_train_zero, users_items_matrix_validate], verbose=1)
  test_result = Deep_AE.evaluate(users_items_matrix_train_avg, users_items_matrix_test)
  print("Fold no: ",f,test_result)

Epoch 1/100
20/20 [==============================] - 16s 647ms/step - loss: 38.0044 - masked_rmse: 4.9657 - masked_rmse_clip_10: 4.7799 - val_loss: 7.8867 - val_masked_rmse: 0.7498 - val_masked_rmse_clip_10: 0.7481
Epoch 2/100
20/20 [==============================] - 13s 665ms/step - loss: 9.9671 - masked_rmse: 1.7110 - masked_rmse_clip_10: 1.6677 - val_loss: 7.3729 - val_masked_rmse: 0.6867 - val_masked_rmse_clip_10: 0.6717
Epoch 3/100
20/20 [==============================] - 13s 657ms/step - loss: 10.0707 - masked_rmse: 1.5642 - masked_rmse_clip_10: 1.5002 - val_loss: 6.7375 - val_masked_rmse: 0.6721 - val_masked_rmse_clip_10: 0.6500
Epoch 4/100
20/20 [==============================] - 13s 658ms/step - loss: 9.4734 - masked_rmse: 1.5428 - masked_rmse_clip_10: 1.4714 - val_loss: 6.4402 - val_masked_rmse: 0.6921 - val_masked_rmse_clip_10: 0.6592
Epoch 5/100
20/20 [==============================] - 13s 661ms/step - loss: 8.0744 - masked_rmse: 1.4676 - masked_rmse_clip_10: 1.4153 - val_l

Epoch 1/100
20/20 [==============================] - 17s 699ms/step - loss: 37.9981 - masked_rmse: 4.9607 - masked_rmse_clip_10: 4.7715 - val_loss: 7.9290 - val_masked_rmse: 0.7599 - val_masked_rmse_clip_10: 0.7571
Epoch 2/100
20/20 [==============================] - 14s 719ms/step - loss: 9.7610 - masked_rmse: 1.6865 - masked_rmse_clip_10: 1.6481 - val_loss: 7.2528 - val_masked_rmse: 0.6562 - val_masked_rmse_clip_10: 0.6500
Epoch 3/100
20/20 [==============================] - 14s 722ms/step - loss: 8.0096 - masked_rmse: 1.3567 - masked_rmse_clip_10: 1.3530 - val_loss: 6.4585 - val_masked_rmse: 0.6066 - val_masked_rmse_clip_10: 0.6038
Epoch 4/100
20/20 [==============================] - 14s 724ms/step - loss: 7.2987 - masked_rmse: 1.3053 - masked_rmse_clip_10: 1.3029 - val_loss: 5.9425 - val_masked_rmse: 0.5912 - val_masked_rmse_clip_10: 0.5884
Epoch 5/100
20/20 [==============================] - 14s 712ms/step - loss: 6.7944 - masked_rmse: 1.2912 - masked_rmse_clip_10: 1.2888 - val_lo

Epoch 1/100
20/20 [==============================] - 19s 702ms/step - loss: 39.5084 - masked_rmse: 5.1122 - masked_rmse_clip_10: 4.9074 - val_loss: 8.1195 - val_masked_rmse: 0.7879 - val_masked_rmse_clip_10: 0.7844
Epoch 2/100
20/20 [==============================] - 11s 533ms/step - loss: 10.3705 - masked_rmse: 1.7447 - masked_rmse_clip_10: 1.6881 - val_loss: 7.2890 - val_masked_rmse: 0.6663 - val_masked_rmse_clip_10: 0.6577
Epoch 3/100
20/20 [==============================] - 14s 710ms/step - loss: 8.1395 - masked_rmse: 1.3825 - masked_rmse_clip_10: 1.3730 - val_loss: 6.3652 - val_masked_rmse: 0.6150 - val_masked_rmse_clip_10: 0.6114
Epoch 4/100
20/20 [==============================] - 14s 706ms/step - loss: 7.2812 - masked_rmse: 1.3245 - masked_rmse_clip_10: 1.3172 - val_loss: 5.7309 - val_masked_rmse: 0.5971 - val_masked_rmse_clip_10: 0.5936
Epoch 5/100
20/20 [==============================] - 13s 667ms/step - loss: 6.8010 - masked_rmse: 1.3201 - masked_rmse_clip_10: 1.3090 - val_l

Epoch 1/100
20/20 [==============================] - 19s 823ms/step - loss: 39.0383 - masked_rmse: 5.0628 - masked_rmse_clip_10: 4.8616 - val_loss: 8.1316 - val_masked_rmse: 0.7781 - val_masked_rmse_clip_10: 0.7744
Epoch 2/100
20/20 [==============================] - 13s 668ms/step - loss: 10.2281 - masked_rmse: 1.7241 - masked_rmse_clip_10: 1.6770 - val_loss: 7.3316 - val_masked_rmse: 0.6550 - val_masked_rmse_clip_10: 0.6489
Epoch 3/100
20/20 [==============================] - 13s 659ms/step - loss: 8.3814 - masked_rmse: 1.3827 - masked_rmse_clip_10: 1.3717 - val_loss: 6.6123 - val_masked_rmse: 0.6090 - val_masked_rmse_clip_10: 0.6044
Epoch 4/100
20/20 [==============================] - 13s 655ms/step - loss: 7.7017 - masked_rmse: 1.3331 - masked_rmse_clip_10: 1.3235 - val_loss: 6.0801 - val_masked_rmse: 0.5835 - val_masked_rmse_clip_10: 0.5816
Epoch 5/100
20/20 [==============================] - 13s 658ms/step - loss: 7.1253 - masked_rmse: 1.3026 - masked_rmse_clip_10: 1.2976 - val_l

Epoch 1/100
20/20 [==============================] - 22s 813ms/step - loss: 37.7994 - masked_rmse: 4.9511 - masked_rmse_clip_10: 4.7685 - val_loss: 7.8629 - val_masked_rmse: 0.7563 - val_masked_rmse_clip_10: 0.7544
Epoch 2/100
20/20 [==============================] - 11s 560ms/step - loss: 9.5422 - masked_rmse: 1.6806 - masked_rmse_clip_10: 1.6435 - val_loss: 6.9315 - val_masked_rmse: 0.6630 - val_masked_rmse_clip_10: 0.6571
Epoch 3/100
20/20 [==============================] - 13s 651ms/step - loss: 7.4470 - masked_rmse: 1.3503 - masked_rmse_clip_10: 1.3488 - val_loss: 5.8375 - val_masked_rmse: 0.6159 - val_masked_rmse_clip_10: 0.6140
Epoch 4/100
20/20 [==============================] - 13s 646ms/step - loss: 6.4351 - masked_rmse: 1.2938 - masked_rmse_clip_10: 1.2936 - val_loss: 5.0709 - val_masked_rmse: 0.6008 - val_masked_rmse_clip_10: 0.5989
Epoch 5/100
20/20 [==============================] - 13s 655ms/step - loss: 5.7071 - masked_rmse: 1.2796 - masked_rmse_clip_10: 1.2794 - val_lo

## Dropout

In [ ]:
import gc

In [ ]:
losses=[]
val_losses=[]
rmses=[]
val_rmses=[]
dropouts=[0,0.5,0.65,0.8]
for d in dropouts:
  gc.collect()
  Deep_AE = Deep_AE_model(users_items_matrix_train_zero,[512, 512, 1024, 512, 512], activation, last_activation, d, regularizer_encode, regularizer_decode)
  Deep_AE.compile(optimizer = Adam(lr=0.0001), loss=masked_mse, metrics=[masked_rmse,masked_rmse_clip_10]) 
  hist_Deep_AE = Deep_AE.fit(x=users_items_matrix_train_zero, y=users_items_matrix_train_zero,
                  epochs=100,
                  batch_size=256,
                  validation_data=[users_items_matrix_train_zero, users_items_matrix_validate], verbose=1)
  test_result = Deep_AE.evaluate(users_items_matrix_train_avg, users_items_matrix_test)
  print(test_result)
  loss = hist_Deep_AE.history['loss']
  val_loss = hist_Deep_AE.history['val_loss']
  rmse = hist_Deep_AE.history['masked_rmse']
  val_rmse = hist_Deep_AE.history['val_masked_rmse']
  losses.append(loss)
  val_losses.append(val_loss)
  rmses.append(rmse)
  val_rmses.append(val_rmse)


Epoch 1/100
20/20 [==============================] - 16s 708ms/step - loss: 21.3455 - masked_rmse: 3.3592 - masked_rmse_clip_10: 3.0670 - val_loss: 6.1642 - val_masked_rmse: 1.2498 - val_masked_rmse_clip_10: 1.2320
Epoch 2/100
20/20 [==============================] - 9s 470ms/step - loss: 5.9308 - masked_rmse: 1.3991 - masked_rmse_clip_10: 1.3841 - val_loss: 5.4181 - val_masked_rmse: 1.1082 - val_masked_rmse_clip_10: 1.1018
Epoch 3/100
20/20 [==============================] - 11s 536ms/step - loss: 5.2907 - masked_rmse: 1.2734 - masked_rmse_clip_10: 1.2695 - val_loss: 5.0990 - val_masked_rmse: 1.0673 - val_masked_rmse_clip_10: 1.0618
Epoch 4/100
20/20 [==============================] - 11s 538ms/step - loss: 4.9699 - masked_rmse: 1.2367 - masked_rmse_clip_10: 1.2320 - val_loss: 4.8261 - val_masked_rmse: 1.0440 - val_masked_rmse_clip_10: 1.0384
Epoch 5/100
20/20 [==============================] - 13s 657ms/step - loss: 4.6250 - masked_rmse: 1.1758 - masked_rmse_clip_10: 1.1706 - val_los

Epoch 1/100
20/20 [==============================] - 16s 685ms/step - loss: 23.1144 - masked_rmse: 3.5785 - masked_rmse_clip_10: 3.2210 - val_loss: 6.5253 - val_masked_rmse: 1.3321 - val_masked_rmse_clip_10: 1.2917
Epoch 2/100
20/20 [==============================] - 12s 624ms/step - loss: 6.1871 - masked_rmse: 1.4724 - masked_rmse_clip_10: 1.4500 - val_loss: 5.5741 - val_masked_rmse: 1.1246 - val_masked_rmse_clip_10: 1.1162
Epoch 3/100
20/20 [==============================] - 12s 618ms/step - loss: 5.5353 - masked_rmse: 1.3275 - masked_rmse_clip_10: 1.3189 - val_loss: 5.2125 - val_masked_rmse: 1.0796 - val_masked_rmse_clip_10: 1.0745
Epoch 4/100
20/20 [==============================] - 12s 614ms/step - loss: 5.1942 - masked_rmse: 1.2787 - masked_rmse_clip_10: 1.2729 - val_loss: 4.9870 - val_masked_rmse: 1.0635 - val_masked_rmse_clip_10: 1.0581
Epoch 5/100
20/20 [==============================] - 12s 620ms/step - loss: 4.8867 - masked_rmse: 1.2319 - masked_rmse_clip_10: 1.2244 - val_lo

Epoch 1/100
20/20 [==============================] - 13s 584ms/step - loss: 23.6691 - masked_rmse: 3.6576 - masked_rmse_clip_10: 3.2858 - val_loss: 6.9737 - val_masked_rmse: 1.4379 - val_masked_rmse_clip_10: 1.3561
Epoch 2/100
20/20 [==============================] - 11s 548ms/step - loss: 6.3915 - masked_rmse: 1.5345 - masked_rmse_clip_10: 1.4981 - val_loss: 5.6866 - val_masked_rmse: 1.1492 - val_masked_rmse_clip_10: 1.1371
Epoch 3/100
20/20 [==============================] - 13s 643ms/step - loss: 5.6657 - masked_rmse: 1.3681 - masked_rmse_clip_10: 1.3571 - val_loss: 5.2688 - val_masked_rmse: 1.0907 - val_masked_rmse_clip_10: 1.0847
Epoch 4/100
20/20 [==============================] - 12s 620ms/step - loss: 5.2998 - masked_rmse: 1.3106 - masked_rmse_clip_10: 1.3031 - val_loss: 5.0388 - val_masked_rmse: 1.0697 - val_masked_rmse_clip_10: 1.0631
Epoch 5/100
20/20 [==============================] - 12s 610ms/step - loss: 5.0214 - masked_rmse: 1.2664 - masked_rmse_clip_10: 1.2581 - val_lo

Epoch 1/100
20/20 [==============================] - 12s 511ms/step - loss: 25.6035 - masked_rmse: 3.8641 - masked_rmse_clip_10: 3.4468 - val_loss: 7.4992 - val_masked_rmse: 1.5397 - val_masked_rmse_clip_10: 1.3857
Epoch 2/100
20/20 [==============================] - 11s 541ms/step - loss: 6.9106 - masked_rmse: 1.6512 - masked_rmse_clip_10: 1.5958 - val_loss: 5.7979 - val_masked_rmse: 1.1600 - val_masked_rmse_clip_10: 1.1482
Epoch 3/100
20/20 [==============================] - 11s 539ms/step - loss: 6.0012 - masked_rmse: 1.4464 - masked_rmse_clip_10: 1.4280 - val_loss: 5.3808 - val_masked_rmse: 1.0901 - val_masked_rmse_clip_10: 1.0833
Epoch 4/100
20/20 [==============================] - 10s 534ms/step - loss: 5.6050 - masked_rmse: 1.3769 - masked_rmse_clip_10: 1.3652 - val_loss: 5.1685 - val_masked_rmse: 1.0763 - val_masked_rmse_clip_10: 1.0695
Epoch 5/100
20/20 [==============================] - 12s 609ms/step - loss: 5.3170 - masked_rmse: 1.3318 - masked_rmse_clip_10: 1.3218 - val_lo

## Number of hidden layers

In [ ]:
losses=[]
val_losses=[]
rmses=[]
val_rmses=[]
depths=[2,4,6,8,10,12]
for d in depths:
  gc.collect()
  Deep_AE = Deep_AE_model(users_items_matrix_train_zero,[128]*d, activation, last_activation,0, regularizer_encode, regularizer_decode)
  Deep_AE.compile(optimizer = Adam(lr=0.0001), loss=masked_mse, metrics=[masked_rmse,masked_rmse_clip_10]) 
  hist_Deep_AE = Deep_AE.fit(x=users_items_matrix_train_zero, y=users_items_matrix_train_zero,
                  epochs=100,
                  batch_size=256,
                  validation_data=[users_items_matrix_train_zero, users_items_matrix_validate], verbose=1)
  test_result = Deep_AE.evaluate(users_items_matrix_train_avg, users_items_matrix_test)
  print(test_result)
  loss = hist_Deep_AE.history['loss']
  val_loss = hist_Deep_AE.history['val_loss']
  rmse = hist_Deep_AE.history['masked_rmse']
  val_rmse = hist_Deep_AE.history['val_masked_rmse']
  losses.append(loss)
  val_losses.append(val_loss)
  rmses.append(rmse)
  val_rmses.append(val_rmse)

Epoch 1/100
20/20 [==============================] - 6s 250ms/step - loss: 43.4301 - masked_rmse: 5.9505 - masked_rmse_clip_10: 5.4504 - val_loss: 13.5996 - val_masked_rmse: 2.8246 - val_masked_rmse_clip_10: 2.5504
Epoch 2/100
20/20 [==============================] - 5s 281ms/step - loss: 12.1652 - masked_rmse: 2.9359 - masked_rmse_clip_10: 2.8183 - val_loss: 4.3903 - val_masked_rmse: 1.4582 - val_masked_rmse_clip_10: 1.4296
Epoch 3/100
20/20 [==============================] - 3s 170ms/step - loss: 4.8478 - masked_rmse: 1.7499 - masked_rmse_clip_10: 1.7382 - val_loss: 3.2621 - val_masked_rmse: 1.2086 - val_masked_rmse_clip_10: 1.1996
Epoch 4/100
20/20 [==============================] - 3s 170ms/step - loss: 3.4712 - masked_rmse: 1.4718 - masked_rmse_clip_10: 1.4643 - val_loss: 2.9472 - val_masked_rmse: 1.1345 - val_masked_rmse_clip_10: 1.1280
Epoch 5/100
20/20 [==============================] - 4s 208ms/step - loss: 2.9686 - masked_rmse: 1.3624 - masked_rmse_clip_10: 1.3570 - val_loss:

Epoch 1/100
20/20 [==============================] - 6s 248ms/step - loss: 40.8219 - masked_rmse: 5.6749 - masked_rmse_clip_10: 5.1843 - val_loss: 6.6763 - val_masked_rmse: 1.8706 - val_masked_rmse_clip_10: 1.7133
Epoch 2/100
20/20 [==============================] - 4s 185ms/step - loss: 5.8304 - masked_rmse: 1.9928 - masked_rmse_clip_10: 1.9388 - val_loss: 3.2583 - val_masked_rmse: 1.2216 - val_masked_rmse_clip_10: 1.2150
Epoch 3/100
20/20 [==============================] - 4s 178ms/step - loss: 3.1073 - masked_rmse: 1.3866 - masked_rmse_clip_10: 1.3822 - val_loss: 2.7986 - val_masked_rmse: 1.0880 - val_masked_rmse_clip_10: 1.0829
Epoch 4/100
20/20 [==============================] - 4s 180ms/step - loss: 2.6703 - masked_rmse: 1.2555 - masked_rmse_clip_10: 1.2543 - val_loss: 2.6171 - val_masked_rmse: 1.0514 - val_masked_rmse_clip_10: 1.0471
Epoch 5/100
20/20 [==============================] - 5s 233ms/step - loss: 2.5100 - masked_rmse: 1.2082 - masked_rmse_clip_10: 1.2077 - val_loss: 2

[31.539810180664062, 4.810515880584717, 1.6836496591567993]
Epoch 1/100
20/20 [==============================] - 6s 215ms/step - loss: 33.3177 - masked_rmse: 4.9135 - masked_rmse_clip_10: 4.4706 - val_loss: 5.7769 - val_masked_rmse: 1.7297 - val_masked_rmse_clip_10: 1.7234
Epoch 2/100
20/20 [==============================] - 3s 173ms/step - loss: 4.5533 - masked_rmse: 1.7066 - masked_rmse_clip_10: 1.6584 - val_loss: 3.3467 - val_masked_rmse: 1.1841 - val_masked_rmse_clip_10: 1.1693
Epoch 3/100
20/20 [==============================] - 4s 192ms/step - loss: 3.0733 - masked_rmse: 1.3132 - masked_rmse_clip_10: 1.3112 - val_loss: 2.8260 - val_masked_rmse: 1.0585 - val_masked_rmse_clip_10: 1.0546
Epoch 4/100
20/20 [==============================] - 4s 224ms/step - loss: 2.8170 - masked_rmse: 1.2344 - masked_rmse_clip_10: 1.2343 - val_loss: 2.7309 - val_masked_rmse: 1.0357 - val_masked_rmse_clip_10: 1.0316
Epoch 5/100
20/20 [==============================] - 4s 181ms/step - loss: 2.7249 - mas

Epoch 1/100
20/20 [==============================] - 6s 211ms/step - loss: 30.2080 - masked_rmse: 4.5593 - masked_rmse_clip_10: 4.1780 - val_loss: 4.8175 - val_masked_rmse: 1.5048 - val_masked_rmse_clip_10: 1.4718
Epoch 2/100
20/20 [==============================] - 6s 299ms/step - loss: 4.2177 - masked_rmse: 1.5712 - masked_rmse_clip_10: 1.5358 - val_loss: 3.2610 - val_masked_rmse: 1.1146 - val_masked_rmse_clip_10: 1.1105
Epoch 3/100
20/20 [==============================] - 4s 184ms/step - loss: 3.2459 - masked_rmse: 1.2926 - masked_rmse_clip_10: 1.2920 - val_loss: 3.0676 - val_masked_rmse: 1.0549 - val_masked_rmse_clip_10: 1.0509
Epoch 4/100
20/20 [==============================] - 4s 186ms/step - loss: 3.0591 - masked_rmse: 1.2388 - masked_rmse_clip_10: 1.2378 - val_loss: 2.9495 - val_masked_rmse: 1.0347 - val_masked_rmse_clip_10: 1.0307
Epoch 5/100
20/20 [==============================] - 5s 241ms/step - loss: 2.9650 - masked_rmse: 1.2207 - masked_rmse_clip_10: 1.2204 - val_loss: 2

Epoch 1/100
20/20 [==============================] - 8s 286ms/step - loss: 31.9067 - masked_rmse: 4.7111 - masked_rmse_clip_10: 4.3094 - val_loss: 4.8909 - val_masked_rmse: 1.4790 - val_masked_rmse_clip_10: 1.4745
Epoch 2/100
20/20 [==============================] - 4s 179ms/step - loss: 4.4836 - masked_rmse: 1.5862 - masked_rmse_clip_10: 1.5494 - val_loss: 3.6059 - val_masked_rmse: 1.1292 - val_masked_rmse_clip_10: 1.1238
Epoch 3/100
20/20 [==============================] - 4s 186ms/step - loss: 3.5244 - masked_rmse: 1.3045 - masked_rmse_clip_10: 1.3038 - val_loss: 3.3030 - val_masked_rmse: 1.0674 - val_masked_rmse_clip_10: 1.0635
Epoch 4/100
20/20 [==============================] - 5s 246ms/step - loss: 3.3253 - masked_rmse: 1.2514 - masked_rmse_clip_10: 1.2510 - val_loss: 3.1922 - val_masked_rmse: 1.0357 - val_masked_rmse_clip_10: 1.0317
Epoch 5/100
20/20 [==============================] - 3s 177ms/step - loss: 3.2025 - masked_rmse: 1.2220 - masked_rmse_clip_10: 1.2218 - val_loss: 3

Epoch 1/100
20/20 [==============================] - 7s 223ms/step - loss: 31.5509 - masked_rmse: 4.6446 - masked_rmse_clip_10: 4.2327 - val_loss: 5.0943 - val_masked_rmse: 1.4682 - val_masked_rmse_clip_10: 1.4623
Epoch 2/100
20/20 [==============================] - 4s 190ms/step - loss: 4.7002 - masked_rmse: 1.5708 - masked_rmse_clip_10: 1.5386 - val_loss: 3.7710 - val_masked_rmse: 1.1118 - val_masked_rmse_clip_10: 1.1074
Epoch 3/100
20/20 [==============================] - 5s 252ms/step - loss: 3.7305 - masked_rmse: 1.2961 - masked_rmse_clip_10: 1.2956 - val_loss: 3.5132 - val_masked_rmse: 1.0471 - val_masked_rmse_clip_10: 1.0425
Epoch 4/100
20/20 [==============================] - 4s 188ms/step - loss: 3.5420 - masked_rmse: 1.2426 - masked_rmse_clip_10: 1.2413 - val_loss: 3.3900 - val_masked_rmse: 1.0239 - val_masked_rmse_clip_10: 1.0193
Epoch 5/100
20/20 [==============================] - 4s 188ms/step - loss: 3.4238 - masked_rmse: 1.2157 - masked_rmse_clip_10: 1.2143 - val_loss: 3